In [1]:
%matplotlib inline
from __future__ import division
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
# pd.options.display.max_rows = 12
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 4))
pd.set_option('display.max_columns', None) #输出所有列

def set_ch():
    from pylab import mpl
    #中文字体问题已经解决，记录在evernote中
    #mpl.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
    mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
set_ch()
from datetime import datetime,timedelta
import matplotlib
matplotlib.style.use('ggplot')
import sys
import json

In [151]:
# import data
df_up = pd.read_csv('data/index_up.csv')
df_middle = pd.read_csv('data/index_mid.csv')
df_down = pd.read_csv('data/index_down.csv')

* visa_typ_cd: 套餐的类型(?)
* group: delete
* min_time: 第一次做作业的时间
* par_days: 实际做作业起始
* actual_day: 实际做作业的天数
* sub_tol: 总的提交次数
* sub_avg = sub_tol / par_days: 提交的平均次数
* sub_avg_a = sub_tol / actual_day: 实际
* ex_tol: 总的做题数
* ex_subrate: 一次按一个package提交，中途出去了就按当前的做题数/package题数。基于此，求整个套餐时间内提交的均值
* sub_ins_sd: 两次提交之间的时间间隔的方差，单位s。超过12H（43200s）就不认为该间隔有意义，所以归0
* ins_sd: 做作业间隔天数的方差
* ins_m: 做作业间隔天数的均值
* auc_sub_avg: 每次提交的平均分数
* auc_sub_sd:每次提交的分数方差
* ins_min: 最少的做作业间隔天数（delete？）
* ins_four_1: 下四分位数（delete？多0情况）
* ins_max: 最大的做作业间隔天数
* ins_summit_min: 两次提交的最小时间间隔	
* ins_summit_four_1: 两次提交的时间间隔的下四分位数
* ins_summit_max: 两次提交的最大时间间隔（单位S）
* 0point_sub: 0点——1点之间的提交次数
* 1point_sub: 1点——2点之间的提交次数
* 0point_ex: 0点——1点之间的做题数量
* 1point_ex: 1点——2点之间的做题数量
* notscWe_sub: 周末的提交次数（不包含寒暑假）
* notscWe_ex: 周末的做作业的题目数量（不包含寒暑假）
* notscWd_sub: 工作日的提交次数（不包含寒暑假）
* notscWd_ex: 工作日的做作业的题目数量（不包含寒暑假）
* sc_sub: 寒暑假的提交次数
* sc_ex: 寒暑假的作业的题目数量
* pub_avg: 老师布置作业后学生隔几天去做的均值
* pub_sd: 老师布置作业后学生隔几天去做的方差
* n_ex: 总的做题数量（已去重）
* n_pkg: 总的package数量（已去重）
* n_pra_ex_avg: 每一道题的练习次数的均值
* n_pra_ex_sd: 每一道题的练习次数的方差
* y: 是否续费
* top_schl_id:学校机构代码
* school:down，up， middle

In [173]:
df_daqiao = df_up[df_up.top_schl_id == 432466]
df_wuhanbb = df_up[df_up.top_schl_id == 437367]
df_only = df_middle[df_middle.top_schl_id == 430552]
df_siqi = df_down[df_down.top_schl_id == 432710]

In [179]:
# 导入类别标签
df_down_pca = pd.read_csv('data/down_pca.csv').drop(['Unnamed: 0'],axis=1)
df_up_pca = pd.read_csv('data/up_pca.csv').drop(['Unnamed: 0'],axis=1)
df_middle_pca = pd.read_csv('data/middle_pca.csv').drop(['Unnamed: 0'],axis=1)

In [187]:
df_daqiao = pd.merge(df_daqiao, df_up_pca[['cluster_label']], left_index=True, right_index=True)
df_wuhanbb = pd.merge(df_wuhanbb, df_up_pca[['cluster_label']], left_index=True, right_index=True)
df_only = pd.merge(df_only, df_middle_pca[['cluster_label']], left_index=True, right_index=True)
df_siqi = pd.merge(df_siqi, df_down_pca[['cluster_label']], left_index=True, right_index=True)

In [197]:
df_4schls = pd.concat([df_daqiao, df_wuhanbb, df_only, df_siqi], axis=0).reset_index().drop(['index'], axis=1)

In [192]:
# 将类别变量编码
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
input_classes = df_4schls.visa_typ_cd.values
label_encoder.fit(input_classes)
print ("\nClass mapping:")
for i, item in enumerate(label_encoder.classes_):
    print (item,'-->', i)
    
labels = df_4schls.visa_typ_cd.values
encoded_labels = label_encoder.transform(labels)
print "\nLabels = ", labels
print "Encoded labels =", list(encoded_labels)


Class mapping:
('A', '-->', 0)
('A1', '-->', 1)
('A2', '-->', 2)
('A3', '-->', 3)
('B', '-->', 4)
('C', '-->', 5)
('D1', '-->', 6)
('D2', '-->', 7)
('D3', '-->', 8)

Labels =  ['A' 'A' 'B' ..., 'D2' 'D2' 'D2']
Encoded labels = [0, 0, 4, 6, 0, 6, 8, 6, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 4, 0, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 0, 4, 4, 4, 0, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 0, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 0, 4, 4, 0, 0, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 0, 0, 4, 4, 4, 0, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 0, 4, 0, 4, 4, 4, 0, 4, 0, 4, 0, 4, 0, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 0, 4, 0, 4, 4, 0, 4, 4, 0, 4, 4, 0, 4, 4, 0, 4, 0, 4, 4, 4, 0, 0, 4, 0, 4, 4, 0, 4, 0, 0, 4, 4, 4, 0, 4, 4, 4, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 4, 4, 0, 4, 0, 4, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 0, 4, 4, 4, 0, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 

In [199]:
# data
df_4schls['visa_typ_cd_encoded'] = encoded_labels
X = df_4schls[['top_schl_id', 'visa_typ_cd_encoded', 'cluster_label']].values
y = df_4schls.y.values

In [259]:
# decision tree:c art
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
import graphviz 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

def cart(X, y):
    # one-hot encoding
    enc = OneHotEncoder()
    enc.fit(X)
    X_new = enc.transform(X).toarray()
    
    # decision tree: cart(gini)
    clf = tree.DecisionTreeClassifier(criterion='gini', max_depth=4, min_samples_leaf=50)
    clf = clf.fit(X_new, y)
    
    # k-fold cross-validation
    scores = cross_val_score(clf, X_new, y, scoring='f1', cv=10)
    cnf_matrix = confusion_matrix(y, clf.predict(X_new))

    print 'F1-score is: %s' % scores
    print 'confusion matrix is: %s' % cnf_matrix
    
    # save the tree img
    dot_data = tree.export_graphviz(clf, out_file=None) 
    graph = graphviz.Source(dot_data) 
    graph.render("img/tree")

In [260]:
cart(X,y)

F1-score is: [ 0.9763  0.9762  0.9645  0.9662  0.9601  0.9624  0.9583  0.7902  0.7192
  0.7886]
confusion matrix is: [[ 26109  15483]
 [ 19537 183724]]


* top_schl_id: 4个类别
* visa_typ_cd: 9个类别
* cluster_lable: 3个类别

In [249]:
col_name = ['430552', '432466', '432710', '437367',
           'A','A1','A2','A3','B','C','D1','D2','D3',
           '0','1','2']

![](img/tree.png)

* X[1]: 表示学校为大桥
* X[14]: 表示为聚类的1类
* X[4]: 表示购买的套餐为A
* X[0]: 学校为昂立
* X[3]: 学校为武汉贝贝
* X[2]: 学校为思奇